In [1]:
import shutil
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon import nn
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
from mxnet.gluon.model_zoo.model_store import _model_sha1
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import h5py
import os
from glob import glob
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

ctx = [mx.cpu()]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Pre-process train images
df = pd.read_csv('labels.csv')
path = 'for_train'

if os.path.exists(path):
    shutil.rmtree(path)

for i, (fname, breed) in df.iterrows():
    path2 = '%s/%s' % (path, breed)
    if not os.path.exists(path2):
        os.makedirs(path2)
    os.symlink('../../train/%s.jpg' % fname, '%s/%s.jpg' % (path2, fname))

In [3]:
#Pre-process test images
df = pd.read_csv('sample_submission.csv')
path = 'for_test'
breed = '0'

if os.path.exists(path):
    shutil.rmtree(path)

for fname in df['id']:
    path2 = '%s/%s' % (path, breed)
    if not os.path.exists(path2):
        os.makedirs(path2)
    os.symlink('../../test/%s.jpg' % fname, '%s/%s.jpg' % (path2, fname))

In [5]:
#Loading pre-saved files if they exist

X_224, X_299, y = nd.load('train.nd')
X_224_test, X_299_test = nd.load('test.nd')

In [6]:
#function for extracting bottleneck features
def save_features(model_name, data_train_iter, data_test_iter, ignore=False):
    if os.path.exists('features_train_%s.nd' % model_name) and ignore:
        if os.path.exists('features_test_%s.nd' % model_name):
            return
    
    net = models.get_model(model_name, pretrained=True, ctx=ctx)
    
    for prefix, data_iter in zip(['train', 'test'], [data_train_iter, data_test_iter]):
        features = []
        for data in tqdm(data_iter):
            for data_slice in gluon.utils.split_and_load(data, ctx, even_split=False):
                feature = net.features(data_slice)
                if 'squeezenet' in model_name:
                    feature = gluon.nn.GlobalAvgPool2D()(feature)
                feature = gluon.nn.Flatten()(feature)
                features.append(feature.as_in_context(mx.cpu()))
            nd.waitall()
        
        features = nd.concat(*features, dim=0)
        nd.save('features_%s_%s.nd' % (prefix, model_name), features)

In [7]:
#Load data to DataLoader
batch_size = 128

data_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_224), batch_size=batch_size)
data_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_299), batch_size=batch_size)

data_test_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_224_test), batch_size=batch_size)
data_test_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_299_test), batch_size=batch_size)

In [8]:
#Extracting bottleneck features
for model in sorted(_model_sha1.keys()):
    print (model)
    if model == 'inceptionv3':
        save_features(model, data_iter_299, data_test_iter_299)
    if model == 'resnet152_v1':
        save_features(model, data_iter_224, data_test_iter_224)

alexnet
densenet121
densenet161
densenet169
densenet201
inceptionv3


100%|██████████| 81/81 [28:26<00:00, 21.06s/it]


mobilenet0.25
mobilenet0.5
mobilenet0.75
mobilenet1.0
resnet101_v1
resnet101_v2
resnet152_v1


100%|██████████| 81/81 [46:56<00:00, 34.77s/it]


resnet152_v2
resnet18_v1
resnet18_v2
resnet34_v1
resnet34_v2
resnet50_v1
resnet50_v2
squeezenet1.0
squeezenet1.1
vgg11
vgg11_bn
vgg13
vgg13_bn
vgg16
vgg16_bn
vgg19
vgg19_bn


In [9]:
#Load Dataset 
df = pd.read_csv('sample_submission.csv')
synset = list(df.columns[1:])

n = len(glob('Images/*/*.jpg'))
X_224 = nd.zeros((n, 3, 224, 224))
X_299 = nd.zeros((n, 3, 299, 299))
y = nd.zeros((n,))

mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

for i, file_name in tqdm(enumerate(glob('Images/*/*.jpg')), total=n):
    img = cv2.imread(file_name)
    img_224 = ((cv2.resize(img, (224, 224))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
    img_299 = ((cv2.resize(img, (299, 299))[:,:,::-1] / 255.0 - mean) / std).transpose((2, 0, 1))
    
    X_224[i] = nd.array(img_224)
    X_299[i] = nd.array(img_299)
    
    y[i] = synset.index(file_name.split('/')[1][10:].lower())
    
    nd.waitall()

100%|██████████| 20580/20580 [03:35<00:00, 95.40it/s]


In [10]:
#Function for acquiring bottleneck features
def get_features(model_name, data_iter):
    net = models.get_model(model_name, pretrained=True, ctx=ctx)
    features = []
    for data in tqdm(data_iter):
        for data_slice in gluon.utils.split_and_load(data, ctx, even_split=False):
            feature = net.features(data_slice)
            feature = gluon.nn.Flatten()(feature)
            features.append(feature.as_in_context(mx.cpu()))
        nd.waitall()

    features = nd.concat(*features, dim=0)
    return features

In [11]:
#Getting bottleneck features from inception and resnet. Then concatenate them.
batch_size = 128

data_iter_224 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_224), batch_size=batch_size)
data_iter_299 = gluon.data.DataLoader(gluon.data.ArrayDataset(X_299), batch_size=batch_size)

model_names = ['inceptionv3', 'resnet152_v1']

features = []
for model_name in model_names:
    if model_name == 'inceptionv3':
        features.append(get_features(model_name, data_iter_299))
    else:
        features.append(get_features(model_name, data_iter_224))
        
features = nd.concat(*features, dim=1)
data_iter_train = gluon.data.DataLoader(gluon.data.ArrayDataset(features, y), batch_size, shuffle=True)

100%|██████████| 161/161 [1:28:14<00:00, 32.89s/it]


In [14]:
# Helper functions for building the model, getting accuracy and evaluation
def build_model():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.BatchNorm())
        net.add(nn.Dense(1024))
        net.add(nn.BatchNorm())
        net.add(nn.Activation('relu'))
        net.add(nn.Dropout(0.5))
        net.add(nn.Dense(120))

    net.initialize(ctx=ctx)
    return net

ctx = mx.cpu()
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

def evaluate(net, data_iter):
    loss, acc, n = 0., 0., 0.
    steps = len(data_iter)
    for data, label in data_iter:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [15]:
#Training the model
net = build_model()

epochs = 100
batch_size = 128
lr_sch = mx.lr_scheduler.FactorScheduler(step=1500, factor=0.5)
trainer = gluon.Trainer(net.collect_params(), 'adam', 
                        {'learning_rate': 1e-3, 'lr_scheduler': lr_sch})

for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    steps = len(data_iter_train)
    for data, label in data_iter_train:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)

        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    print("Epoch %d. loss: %.4f, acc: %.2f%%" % (epoch+1, train_loss/steps, train_acc/steps*100))

Epoch 1. loss: 0.6046, acc: 83.94%
Epoch 2. loss: 0.2498, acc: 91.88%
Epoch 3. loss: 0.1827, acc: 94.05%
Epoch 4. loss: 0.1484, acc: 95.01%
Epoch 5. loss: 0.1202, acc: 95.81%
Epoch 6. loss: 0.1044, acc: 96.63%
Epoch 7. loss: 0.0861, acc: 97.04%
Epoch 8. loss: 0.0782, acc: 97.43%
Epoch 9. loss: 0.0634, acc: 97.87%
Epoch 10. loss: 0.0526, acc: 98.19%
Epoch 11. loss: 0.0352, acc: 98.86%
Epoch 12. loss: 0.0316, acc: 99.04%
Epoch 13. loss: 0.0288, acc: 99.20%
Epoch 14. loss: 0.0254, acc: 99.33%
Epoch 15. loss: 0.0254, acc: 99.25%
Epoch 16. loss: 0.0240, acc: 99.36%
Epoch 17. loss: 0.0228, acc: 99.44%
Epoch 18. loss: 0.0238, acc: 99.34%
Epoch 19. loss: 0.0207, acc: 99.36%
Epoch 20. loss: 0.0163, acc: 99.58%
Epoch 21. loss: 0.0166, acc: 99.60%
Epoch 22. loss: 0.0148, acc: 99.65%
Epoch 23. loss: 0.0145, acc: 99.63%
Epoch 24. loss: 0.0142, acc: 99.63%
Epoch 25. loss: 0.0130, acc: 99.66%
Epoch 26. loss: 0.0139, acc: 99.60%
Epoch 27. loss: 0.0142, acc: 99.59%
Epoch 28. loss: 0.0132, acc: 99.64%
E

In [16]:
#Load previous generated test bottleneck features
features_test = [nd.load('features_test_%s.nd' % model_name)[0] for model_name in model_names]
features_test = nd.concat(*features_test, dim=1)

In [17]:
#Predict and then generate csv file
output = nd.softmax(net(features_test.as_in_context(ctx))).asnumpy()
df_pred = pd.read_csv('sample_submission.csv')

for i, c in enumerate(df_pred.columns[1:]):
    df_pred[c] = output[:,i]

df_pred.to_csv('pred.csv', index=None)